In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss
from torchvision import transforms

from PIL import Image

#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from sklearn import metrics

import numpy as np
import pandas as pd

In [2]:
table = "tabelaAlunosPPI.xls"
df = pd.read_excel(table)

# Lista de tuplas que contem o caminho da imagem e a classe
xy = []

for index, row in df.iterrows():
    if row['anoIngresso'] == 2023 and row['parecerFinal'] != 'Ignorar (ex-aluno)':
        # Se aluno pertencer a classe PPI, adiciona a lista
        if row['parecerFinal'] == 'Consistente':
            aux = (row['foto'], 1)
            xy.append(aux)
        # Se o aluno for branco, ignora
        else:
            continue

In [3]:
print(xy[0])

('231220171.jpg', 1)


In [4]:
class CustomDataset(Dataset):
    def __init__(self, xy, transform=None):
        self.xy = xy
        self.transform = transform
    
    def __getitem__(self, index):
        name = self.xy[index][0]
        label = self.xy[index][1]
        image = Image.open("C:\\Users\\DELL\\Desktop\\IC\\2023_PPI\\" + name)
        image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.xy)

In [ ]:
#imagem = mpimg.imread("C:\\Users\\DELL\\Desktop\\IC\\2023_PPI\\231025327.jpg")
#plt.imshow(imagem)
#plt.axis('off')
#plt.show()

In [5]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

In [6]:
train_data, val_data = train_test_split(xy, test_size=0.1, random_state=40)
#train_data, test_data = train_test_split(xy, test_size=0.2, random_state=40)
#train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=40)

custom_train = CustomDataset(train_data, transform)
#custom_test = CustomDataset(test_data, transform)
custom_val = CustomDataset(val_data, transform)

batch_size = 16
train_loader = DataLoader(custom_train, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(custom_test, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(custom_val, batch_size=batch_size, shuffle=True, drop_last=True)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.leaky_relu = nn.LeakyReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.leaky_relu(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.leaky_relu(x)
        x = self.maxpool3(x)

        x = x.view(-1, 128 * 4 * 4)

        x = self.fc(x)
        x = self.leaky_relu(x)

        x = self.fc2(x)
        x = torch.sigmoid(x)

        return x

In [8]:
def quantile_loss(preds, target, quantile):
    assert 0 < quantile < 1, "Quantile should be in (0, 1) range"
    errors = target - preds
    loss = torch.max((quantile - 1) * errors, quantile * errors)
    return torch.abs(loss).mean()

In [9]:
model = CNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#criterion = nn.BCELoss()
n_epochs = 1

model.train()
for epoch in range(n_epochs):
    for inputs, targets in train_loader:
        output = model(inputs)
        loss = quantile_loss(output.squeeze(), targets.float(), quantile=0.99)
        #loss = criterion(output.squeeze(), targets.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss)

    with torch.no_grad():
        model.eval()
        pred = []
        true = []
        for inputs, targets in val_loader:
            output = model(inputs)
            output = torch.round(output).int()
            pred.append(output.flatten())
            true.append(targets)

        pred = np.array(pred)
        pred = pred.flatten()
        true = np.array(true)
        true = true.flatten()
            
        acc = metrics.accuracy_score(y_true=true, y_pred=pred)

        print(f'Val Accuracy: {acc}')

tensor(0.5114, grad_fn=<MeanBackward0>)
tensor(0.4071, grad_fn=<MeanBackward0>)
tensor(0.1519, grad_fn=<MeanBackward0>)
tensor(0.0023, grad_fn=<MeanBackward0>)
tensor(4.9420e-06, grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tens

In [ ]:
# with torch.no_grad():
#     correct = 0
#     pred = []
#     true = []
#     model.eval()
#     for inputs, targets in test_loader:
#         output = model(inputs)
#         output = torch.round(output).int()
#         pred.append(output.flatten())
#         true.append(targets)

#     pred = np.array(pred)
#     pred = pred.flatten()
#     true = np.array(true)
#     true = true.flatten()
    
# acc = metrics.accuracy_score(true, pred)

# print(f'Test Accuracy: {acc}')

In [10]:
image = Image.open("C:\\Users\\DELL\\Desktop\\IC\\2023_brancos\\231025327.jpg" )
image = image.convert('RGB')
image = transform(image)
image = torch.reshape(image, (1, image.shape[0], image.shape[1], image.shape[2]))

print(image.shape)
branco = 0
with torch.no_grad():
    model.eval()
    pred = []
    true = []
    output = model(image)
    output = int(torch.round(output))
    print(output)
    # if float(output.item()) > 0.5:
    #     output.item() = 1
    # else:
    #     output.item() = 0
    
    true.append(branco)
    pred.append(output)
    acc = metrics.accuracy_score(true, pred)
    print(f'Test Accuracy: {acc}')

torch.Size([1, 3, 32, 32])
1
Test Accuracy: 0.0
